#PUC-RJ - Trabalho de conclusão de curso - Web scraping com filtro baseado em BERT
Data: 03/04/2022
Alunos: Andrius Lopes do Nascimento / Yasmin Teixeira


In [ ]:
pip install sentence-transformers

     |████████████████████████████████| 79 kB 2.5 MB/s 
     |████████████████████████████████| 3.8 MB 29.0 MB/s 
     |████████████████████████████████| 1.2 MB 60.5 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 6.5 MB 66.5 MB/s 
     |████████████████████████████████| 596 kB 74.4 MB/s 
     |████████████████████████████████| 895 kB 58.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=2b2b1680cca4fec18b55a1522bbbfe16837bc08ba518306d26ec637ac1057d5b
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

#Bibliotecas utilizadas

from urllib.request import urlopen                # Biblioteca que permite fazer a solicitação de acesso de site(HTTP) e obtém a resposta
from bs4 import BeautifulSoup                     # Transforma a resposta obtida do url open em um objeto navegável de modo mais simples
import pandas as pd                               # Ocorrerá seu uso na criação do Dataframe que abrigará os dados raspados de cada notícia
import datetime
from datetime import date,datetime
from google.colab import data_table

from sentence_transformers import SentenceTransformer, util
from transformers import BertTokenizer, BertModel

#1 - Raspando os dados desejados 
url = 'https://www.gov.br/mme/pt-br/assuntos/noticias'    #variável que armazena o site a ser acessado
response = urlopen(url)                          # Abre o site desejado e armazena
html = response.read()                           # Lê o conteúdo da resposta do servidor
soup = BeautifulSoup(html, 'html.parser')        #Criação do objeto Beautifulsoup 

#2 - Obtendo as informações desejadas de todas as páginas do site e salvando em dataframe com data e hora
#Esta etapa percorre todas as notícias de todas as páginas de um site usando um laço de for

#Criação de listas
data = []
titulo = []
descricao = []
link = []

contador_paginas=0
cont_noticia = 0   #pertence ao contador de notícias - segundo for
#Início da manipulação de página
ultima_pagina = soup.find('a',{'class':'pagina'}).text  # identifica a última página do site  
num_ultima_pagina = int(ultima_pagina)                  # o resultado obtido a raspagem é uma string. Neecessário converter para inteiro para usar no for   
num_ultima_pagina = num_ultima_pagina+1                 # Para casar com a contagem da página neste site (começa em zero e acaba 1 unidade antes)

cont=0

#for i in range(0,2):                                    #itera em cada página do site
for i in range(0,num_ultima_pagina- 1):               # define início e fim do laço de for
  cont= i*20                                            #emulação da contagem de páginas incrmentando de 20 em 20 - começa em 0 e vai até 3160, por isso o for inicia em 0 e o final subtrai 1 
  #print(cont)                                          #ao invés de iniciar em 1 e terminar em num_ultima_pagina    
  url_pag = f'https://www.gov.br/mme/pt-br/assuntos/noticias?b_start:int={cont}'
  
  #print(url_pag)

  contador_paginas =contador_paginas+1
  response = urlopen(url_pag)                           # Abre o site desejado e armazena
  
  #Tratamento do HTML
  html = response.read()                                # Lê o conteúdo da resposta do servidor
  html = html.decode('utf-8')
  html.split()                                          #Eliminando caracteres de tabulação e quebra de linha - separa por espaço entre caracteres
  " ".join(html.split())                                #Eliminando espaço em branco - o dado precisa estar convertido em string
  " ".join(html.split()).replace('> <', '><')           #Eliminando espaço em branco entre as tags > <   
  soup = BeautifulSoup(html, 'html.parser')             #Criação do objeto Beautifulsoup      
  #print(url_pag)                                       #apresenta o link das páginas varridas  
  
  #Tratamento das strings alocadas em cada rótulo (data, título, descrição e link):
  noticias = soup.find('ul', {'class':'noticias listagem-noticias-com-foto'})               #armazena o conteúdo de toda a ul - #----Início do laço para varrer cada notícia em uma página----------                                                       
  for noticia in noticias.find_all('li'):                                                   #itera cada notícia no conjunto de notícias existentes em uma página
    
    data.append(noticia.find('span', {'class':'data'}).get_text().strip().replace('-',''))  #data da notícia - remove "-" e "/n" antes e depois do conteúdo de interesse
                                                                        
    titulo.append(noticia.find('h2', {'class':'titulo'}).get_text().strip())                #armazena o título, removendo "n"  
    
    scrap_descricao_tratado = noticia.find('span', {'class':'descricao'}).get_text().strip().replace('    ','') #descricao,remove "\n" e espaços    
    ##descricao_noticia = scrap_descricao_tratado.replace('\n',"")                          #remove \n no meio do texto
    ##descricao.append(descricao_noticia)
    descricao.append(scrap_descricao_tratado.replace('\n',""))
    
    link.append(noticia.find(href=True)['href'])                                            #armazena link - não possuin "\n", não precisa strip()
    cont_noticia = cont_noticia+1

  #Criação do Dataframe com definição dos rótulos
  df = pd.DataFrame({
          "Data": data,
          "Titulo":titulo, 
          "Descricao": descricao,
          "Link": link      
      })
  
#Inserção de data e hora no nome do arquivo
hoje = date.today()
hoje_data_hora = datetime.now().isoformat(timespec='seconds')
nome_arq_data_hora = hoje_data_hora.replace(":","-")

#Geração do arquivo em Excel
arquivo_noticias = pd.DataFrame(data = df)
arquivo_noticias.to_excel(f'noticias{nome_arq_data_hora}.xlsx',index=True ,encoding='utf-8')  

print('Número de notícias percorridas',cont_noticia)
print('Numero de páginas percorridas',contador_paginas)
print('número da última página',ultima_pagina)
print('Dia da extração dos dados: ',hoje,"\n")



#3 - Visualização do dataframe como tabela filtrável 
data_table.enable_dataframe_formatter()   #Permite visualizar o dataframe como uma tabela filtrável
arquivo_noticias.head(23)

#print(df)



#4 - Fatiando o Dataframe - colunas Título e Descrição
#key_word = []     #Palavras chave definidas pelo usuário
#noticias = []     #Palavras que formam as notícias do dataframe
filter_words = [] #Palavras filtradas pelo nível de similaridade acima de um determinado valor
similaridade_filter_words = []

#Listas provenientes do dataframe original usadas na similaridade
data_noticia = list(df['Data'])
titulo_noticia = list(df['Titulo'])
descricao_noticia = list(df['Descricao'])
link_noticia = list(df['Link'])

df1 = df.loc[:'Data','Titulo':'Descricao'] #Retorna um dataframe com as colunas escolhidas
#print(df1)

#Visualização do dataframe como tabela filtrável 
data_table.enable_dataframe_formatter()   #Permite visualizar o dataframe como uma tabela filtrável
df1.head(23)                                     #   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#print (df1)

#Capta as informações de título das notícias para avaliar a similaridade ente palavra chave e as notícias
list_titulo = list(df1['Titulo'])                #Transformou a coluna do dataframe de interesse em lista para usar na análise de similaridade
#list_data = list(df1['Data'])

#7 - Usando Transformers na análise de similaridade entre palavra chave e título de notícias

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') 
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")

palavras_chave = ['combustível','gasolina', 'etanol', 'biodiesel','anidro','petróleo','energia','fertilizante','guerra','abastecimento de combustível'
                    'crédito de carbono','petrobras','consumo de eletricidade','gás natural','combustível de aviação']     #Lista com palavras chave definidas pelo usuário
#noticias = ["A gasolina está cara", "hoje vamos berber uma", "a usina vai para de ceder eletricidade"]           #Lista com as notícias oriundas do dataframe


list_data_filtrada = []
list_noticia_filtrada = []                #Lista p/armazenar o titulo da notícia
list_descricao_filtrada = []
list_link_filtrado = []

for palavra in palavras_chave:

  cont_list = 0                                               #será usado para chamar as informações complementares alocadas nas outras colunas
  #for noticia in noticias:
  for noticia in list_titulo:
    #Sentences are encoded by calling model.encode()
    palavra_escolhida = model.encode(palavra)                 #armazena a palavra chave analisada no momento da iteração
  
    palavra_analisada = model.encode(noticia)

    cos_sim = util.cos_sim(palavra_escolhida, palavra_analisada)
    #print("Cosine-Similarity:", cos_sim)

    ##print("valor de similaridade - todos os casos: ",palavra,"--",noticia,"--",cos_sim,"\n")

    valor_sim = 0.400
    if cos_sim >= valor_sim:

      list_data_filtrada.append(data_noticia[cont_list])
      list_noticia_filtrada.append(noticia)                        #título da notícia
      list_descricao_filtrada.append(descricao_noticia[cont_list])
      list_link_filtrado.append(link_noticia[cont_list])

      #print(noticia)    #apresenta o vetor da noticia salva
      #print("valor de similaridade armazenados: ",palavra,"--",noticia,"--",cos_sim,"\n")    #apresenta o vetor da noticia salv
      #print(data_noticia[cont_list], titulo_noticia[cont_list], descricao_noticia[cont_list], link_noticia[cont_list],"\n")

      df_filtrados = pd.DataFrame((zip(list_data_filtrada,list_noticia_filtrada,list_descricao_filtrada,list_link_filtrado)), columns = ['Data_filtrada', 'Titulo_filtrado',
                                      'Descricao filtrada','Link filtrado'])


    cont_list+=1


##df_filtrados = pd.DataFrame((zip(list_data_filtrada,list_noticia_filtrada,list_descricao_filtrada,list_link_filtrado)), columns = ['Data_filtrada', 'Titulo_filtrado',
##                                'Descricao filtrada','Link filtrado'])
##print(df_filtrados)


#Geração do arquivo em Excel
arquivo_noticias = pd.DataFrame(data = df_filtrados)
arquivo_noticias.to_excel(f'noticias_filtradas_{nome_arq_data_hora}.xlsx',index=True ,encoding='utf-8')  



#zzz - Visualização do dataframe como tabela filtrável 
data_table.enable_dataframe_formatter()   #Permite visualizar o dataframe como uma tabela filtrável
df_filtrados.head(15)

#print(df)


Número de notícias percorridas 3217
Numero de páginas percorridas 161
número da última página 161
Dia da extração dos dados:  2022-04-03 



,Data_filtrada,Titulo_filtrado,Descricao filtrada,Link filtrado
0,06/07/2021,Comitê realiza seminário sobre estoques de com...,06/07/2021 -Evento contou com participação de ...,https://www.gov.br/mme/pt-br/assuntos/noticias...
1,10/06/2021,Abastecimento de etanol combustível,10/06/2021 -,https://www.gov.br/mme/pt-br/assuntos/noticias...
2,26/06/2020,Abastecimento de biodiesel,26/06/2020 -,https://www.gov.br/mme/pt-br/assuntos/noticias...
3,19/06/2020,NOTA,19/06/2020 -,https://www.gov.br/mme/pt-br/assuntos/noticias...
4,13/05/2020,Petrobras reduz preço do gás natural,13/05/2020 -,https://www.gov.br/mme/pt-br/assuntos/noticias...
5,10/03/2020,teste,10/03/2020 -,https://www.gov.br/mme/pt-br/assuntos/noticias...
6,12/04/2019,Nota à imprensa: preços de combustíveis,12/04/2019 -,https://www.gov.br/mme/pt-br/assuntos/noticias...
7,02/04/2019,NOTA,02/04/2019 -,https://www.gov.br/mme/pt-br/assuntos/noticias...
8,02/04/2019,NOTA,02/04/2019 -,https://www.gov.br/mme/pt-br/assuntos/noticias...
9,15/05/2017,Combustível Brasil consolida propostas para o ...,15/05/2017 -,https://www.gov.br/mme/pt-br/assuntos/noticias...
